In [ ]:
import cv2
import cv2 as cv
from mmpose.apis import (inference_top_down_pose_model, init_pose_model,
                         vis_pose_result, process_mmdet_results)
from mmdet.apis import inference_detector, init_detector
import mmcv
import numpy as np


# 导入可视化工具包 matplotlib，并让绘制的图像嵌入在 notebook 中
import matplotlib.pyplot as plt
%matplotlib inline

# 定义可视化图像函数，输入图像路径，可视化图像
def show_img_from_path(img_path):
    '''opencv 读入图像，matplotlib 可视化格式为 RGB，因此需将 BGR 转 RGB，最后可视化出来'''
    img = cv2.imread(img_path)
    img_RGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img_RGB)
    plt.show()
    
# 定义可视化图像函数，输入图像 array，可视化图像
def show_img_from_array(img):
    '''输入 array，matplotlib 可视化格式为 RGB，因此需将 BGR 转 RGB，最后可视化出来'''
    img_RGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img_RGB)
    plt.show()


pose_config = 'mmpose/configs/res101_pig.py'
pose_checkpoint = 'mmpose/work_dirs/resnet101_pig/epoch_190.pth'
det_config = 'mmdetection/configs/yolox/yolox_s_8x8_300e_coco.py'
det_checkpoint = 'mmdetection/work_dirs/yolox_s_300e_coco/best_bbox_mAP_epoch_295.pth'

# initialize pose model
pose_model = init_pose_model(pose_config, pose_checkpoint,device='cuda:0')
# initialize detector
det_model = init_detector(det_config, det_checkpoint, device='cuda:0')
point_dic=pose_model.cfg.dataset_info.keypoint_info

In [2]:
skl_dic=pose_model.cfg.dataset_info.skeleton_info

In [3]:
img = '/home/zhenbang/project/openmmlab/mmpose/data/pig_pose/images/p121_6933.jpg'

# inference detection
mmdet_results = inference_detector(det_model, img)

# extract person (COCO_ID=1) bounding boxes from the detection results
pig_results = process_mmdet_results(mmdet_results, cat_id=1)

/home/zhenbang/anaconda3/envs/mmlab/lib/python3.8/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484810403/work/aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [4]:
h,w,c=cv.imread(img).shape
thr=w/5
l=int(thr)
r=int(w-thr)
pig_results = [{'bbox': np.array([l, 0,r-l, h])}]

# inference pose
pose_results, returned_outputs = inference_top_down_pose_model(
    pose_model,
    img,
    pig_results,
    # bbox_thr=0.8,
    format='xyxy',
    dataset=pose_model.cfg.data.test.type)

In [5]:
points=pose_results[0]['keypoints']

In [ ]:
link=list()
for i in range(4):
    l=list()
    l.append(points[0][:2])
    l.append(points[i*4+1:i*4+5,:2])
    link.append(l)



In [ ]:
link

In [ ]:
angle_points=list()
for it in link:
    l=list()
    for i in range(3):
        l.append(np.append(it[0],it[1][i]))
        l.append(np.append(it[1][i+1],it[1][i]))
    l.append(np.append(it[1][0],it[1][1]))
    l.append(np.append(it[1][2],it[1][1]))
    l.append(np.append(it[1][1],it[1][2]))
    l.append(np.append(it[1][3],it[1][2]))
    angle_points.append(l)
angle_points


In [9]:
import math
def angle(v1, v2):
  dx1 = v1[2] - v1[0]
  dy1 = v1[3] - v1[1]
  dx2 = v2[2] - v2[0]
  dy2 = v2[3] - v2[1]
  angle1 = math.atan2(dy1, dx1)
  angle1 = int(angle1 * 180/math.pi)
  # print(angle1)
  angle2 = math.atan2(dy2, dx2)
  angle2 = int(angle2 * 180/math.pi)
  # print(angle2)
  if angle1*angle2 >= 0:
    included_angle = abs(angle1-angle2)
  else:
    included_angle = abs(angle1) + abs(angle2)
  return included_angle

In [ ]:
for it in angle_points:
    for i in range(5):

        print(angle(it[i*2],it[i*2+1],True))

In [ ]:
angle_points

In [ ]:
angles=list()
for it in angle_points:
    l=list()
    for i in range(6):
        l.append(angle(it[i*2],it[i*2+1]))
    angles.append(l)


In [ ]:
angles

In [8]:
def angle_for_model(result):
    points=result['keypoints']
    link=list()
    for i in range(4):
        l=list()
        l.append(points[0][:2])
        l.append(points[i*4+1:i*4+5,:2])
        link.append(l)

    angle_points=list()
    for it in link:
        l=list()
        for i in range(3):
            l.append(np.append(it[0],it[1][i]))
            l.append(np.append(it[1][i+1],it[1][i]))
        l.append(np.append(it[1][0],it[1][1]))
        l.append(np.append(it[1][2],it[1][1]))
        l.append(np.append(it[1][1],it[1][2]))
        l.append(np.append(it[1][3],it[1][2]))
        angle_points.append(l)
        
    angles=list()
    for it in angle_points:
        l=list()
        for i in range(5):
                l.append(angle(it[i*2],it[i*2+1]))
        angles.append(l)
    return angles

In [12]:
from utils import get_angles
get_angles(pose_results[0]['keypoints'])

[[145, 134, 152, 160, 191],
 [136, 155, 180, 189, 200],
 [250, 251, 257, 193, 198],
 [228, 245, 241, 205, 187]]

In [10]:
angle_for_model(pose_results[0])

[[145, 134, 152, 160, 191],
 [136, 155, 180, 189, 200],
 [250, 251, 257, 193, 198],
 [228, 245, 241, 205, 187]]

In [ ]:
vis_result = vis_pose_result(
    pose_model,
    img,
    pose_results,
    dataset=pose_model.cfg.data.test.type,
    show=False)
show_img_from_array(vis_result)